In [ ]:
import json
import os
import math
import gc

In [ ]:
listofReadFile = []
N = 0
for file in os.listdir("LegalCorpus"):
    if file.endswith(".txt"):
        listofReadFile.append(file)
        N += 1

In [ ]:
# Loading the Unique word corpus
with open('UniqueBowOfDocForIDFCorpus.json', 'r') as fp:
    readUniqueWordFromDoc = json.load(fp)

In [ ]:
unionBAgOfWords = []
totalWords = 0
for eachDoc in listofReadFile:
    unionBAgOfWords = set(unionBAgOfWords).union(set(readUniqueWordFromDoc[eachDoc]))

In [ ]:
dynamicDictionary = {}
dynamicDictionary = dict.fromkeys(unionBAgOfWords,0)

In [ ]:
dictionaryIDF = {}
dictionaryWordIDF = {}
listofReadFile = sorted(listofReadFile)
countFile = 1
print ('Building...')
for fileName in listofReadFile:
    uniqueWordList = []
    uniqueWordList = readUniqueWordFromDoc[fileName]
    
    dictionaryWordIDF = dict.fromkeys(set(uniqueWordList), 0)
    for word in dictionaryWordIDF:
        if(dynamicDictionary[word] != 0):
            dictionaryWordIDF[word] = dynamicDictionary[word]
            continue
        for file in listofReadFile:
            doc = []
            doc = readUniqueWordFromDoc[file]
            if(word in doc):
                dictionaryWordIDF[word] += 1
        dynamicDictionary[word] = dictionaryWordIDF[word]
    
    #calculating weighted the 1 + log10(N/df)
    for term, value in dictionaryWordIDF.items():
        dictionaryWordIDF[term] = math.log10(N/float(value))
    dictionaryIDF[fileName] = dictionaryWordIDF
    dictionaryWordIDF = {}
    
    print (countFile, '>', fileName, end = '  ')
    if(countFile % 10 == 0):
        print('\n')
    countFile += 1
    
    # collecting the garbages which takes up memory. Helps to free up memory ==> Execution is Faster.
    gc.collect()

In [ ]:
#saving dictionary to .json file 
with open('IDFCorpus.json', 'w') as fp:
    json.dump(dictionaryIDF, fp, sort_keys=True, indent=0)
fp.close()


#splitting the large file into two
dictionaryIDF13 = {}
dictionaryIDF18 = {}

for key in dictionaryIDF:
    if(key.startswith('19') or key.startswith('2010') or key.startswith('2011') or key.startswith('2012') or key.startswith('2013')):
        dictionaryIDF13[key] = dictionaryIDF[key]
    else:
        dictionaryIDF18[key] = dictionaryIDF[key]

with open('IDFCorpus13.json', 'w') as fp:
    json.dump(dictionaryIDF13, fp, sort_keys=True, indent=0)
fp.close()

with open('IDFCorpus18.json', 'w') as fp:
    json.dump(dictionaryIDF18, fp, sort_keys=True, indent=0)
fp.close()

print ("Build Complete...")